2. Compare the model with DenseNet model
Load CIFAR-10 data set

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10

# Load the CIFAR-10 dataset
(train_images_temp, train_labels_temp), (test_images_temp, test_labels_temp) = cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images_temp, test_images_temp = train_images_temp / 255.0, test_images_temp / 255.0

# Print dataset shapes
print("Train images shape:", train_images_temp.shape)
print("Train labels shape:", train_labels_temp.shape)
print("Test images shape:", test_images_temp.shape)
print("Test labels shape:", test_labels_temp.shape)

#merge data to a common data set

import numpy as np

images = np.concatenate((train_images_temp,test_images_temp),axis=0)
lables = np.concatenate((train_labels_temp,test_labels_temp),axis=0)

num_train_images = int(images.shape[0]*0.6)
num_validate_images = int(images.shape[0]*0.2)
num_test_images = int(images.shape[0]*0.2)

train_images,validate_images,test_images = np.split(images,[num_train_images,num_validate_images+num_test_images])
train_labels,validate_labels,test_labels = np.split(lables,[num_train_images,num_test_images+num_validate_images])



Train images shape: (50000, 32, 32, 3)
Train labels shape: (50000, 1)
Test images shape: (10000, 32, 32, 3)
Test labels shape: (10000, 1)


Import the DenseNet Model and train the model

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121  
from tensorflow.keras.layers import Input


#change the new input shape
new_input_shape = (32,32,3)

#new input layer
new_input_layer = Input(shape=new_input_shape)
# Load the pre-trained DenseNet model 
densenet_model = DenseNet121(weights='imagenet', include_top=True,input_tensor = new_input_layer)

# Optionally, set the model to evaluation mode
densenet_model.trainable = True

# View a summary of the model's architecture
densenet_model.summary()